IMPORTS

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os

import polars as pl

from sentence_transformers import SentenceTransformer, util

from sklearn.metrics import DistanceMetric
import numpy as np

import matplotlib.pyplot as plt

SETTING PATH

In [7]:
# Change working directory to your specific folder
os.chdir('/content/drive/MyDrive/YouTube_Semantic_Search_ML/Data Science')

LOAD DATA

In [8]:
df = pl.read_parquet('data/video-transcripts.parquet')
df_eval = pl.read_csv('data/eval-raw.csv')
df.head()

video_id,datetime,title,description,transcript
str,datetime[μs],str,str,str
"""7Oy2NmPwJXo""",2024-09-26 23:24:35,"""I Quit My Job… Here’s How Much…","""Here, I share all 9 ways I scr…","""14 months ago I made a big lif…"
"""ZVVkdXHqEuM""",2024-09-23 15:45:12,"""Knowledge Distillation Explain…","""""","""knowledge distillation explain…"
"""reXoKNC_Wx4""",2024-09-20 18:15:44,"""Quantization Explained in 60 S…","""""","""here's quantization explained …"
"""9joIFeKuf04""",2024-09-16 14:01:44,"""Python Explained in 60 Seconds…","""""","""here's python explained in 60 …"
"""pNg2DJ4spXg""",2024-09-12 16:36:12,"""Python QuickStart for People L…","""Here, I share a beginner Quick…","""python is the go-to programmin…"


In [9]:
df_eval.head()

query,video_id
str,str
"""ai consulting""","""INlCLmWlojY"""
"""fine tuning llm""","""eC6Hd1hFvos"""
"""When do you recommend fine tun…","""eC6Hd1hFvos"""
"""llm from scratch""","""ZLbVdvOoTKM"""
"""What if you could make a small…","""ZLbVdvOoTKM"""


EMBED TITLES AND TRANSCRIPTS

In [10]:
# define "parameters"
column_to_embed_list = ['title', 'transcript']
model_name_list = ["all-MiniLM-L6-v2", "multi-qa-distilbert-cos-v1", "multi-qa-mpnet-base-dot-v1"]

In [11]:
# generate embeddings for each combination of column and model

# initialize dict to keep track of all text embeddings
text_embedding_dict = {}

for model_name in model_name_list:

    #define embedding model
    model = SentenceTransformer(model_name)

    for column_name in column_to_embed_list:

        # define text embedding identifier
        key_name = model_name + "_" + column_name
        print(key_name)

        # generate embeddings for text under column_name
        %time embedding_arr = model.encode(df[column_name].to_list())
        print('')

        # append embeddings to dict
        text_embedding_dict[key_name] = embedding_arr

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

multi-qa-mpnet-base-dot-v1_title
CPU times: user 7.57 s, sys: 49 ms, total: 7.62 s
Wall time: 7.74 s

multi-qa-mpnet-base-dot-v1_transcript
CPU times: user 2min 57s, sys: 1min 3s, total: 4min
Wall time: 4min 2s



EMBED QUERIES

In [12]:
query_embedding_dict = {}

for model_name in model_name_list:

    #define embedding model
    model = SentenceTransformer(model_name)
    print(model_name)

    # embed query text
    %time embedding_arr = model.encode(df_eval['query'].to_list())
    print('')

    # append embedding to dict
    query_embedding_dict[model_name] = embedding_arr

all-MiniLM-L6-v2
CPU times: user 999 ms, sys: 48 ms, total: 1.05 s
Wall time: 1.1 s

multi-qa-distilbert-cos-v1
CPU times: user 3.93 s, sys: 200 ms, total: 4.13 s
Wall time: 4.15 s

multi-qa-mpnet-base-dot-v1
CPU times: user 7.67 s, sys: 142 ms, total: 7.81 s
Wall time: 7.85 s



EVALUATE SEARCH METHODS

In [13]:
def returnVideoID_index(df: pl.dataframe.frame.DataFrame, df_eval: pl.dataframe.frame.DataFrame, query_n: int) -> int:
    """
        Function to return the index of a dataframe corresponding to the nth row in evaluation dataframe
    """

    return [i for i in range(len(df)) if df['video_id'][i]==df_eval['video_id'][query_n]][0]

In [14]:
def evalTrueRankings(dist_arr_isorted: np.ndarray, df: pl.dataframe.frame.DataFrame, df_eval: pl.dataframe.frame.DataFrame) -> np.ndarray:
    """
        Function to return "true" video ID rankings for each evaluation query
    """

    # intialize array to store rankings of "correct" search result
    true_rank_arr = np.empty((1, dist_arr_isorted.shape[1]))

    # evaluate ranking of correct result for each query
    for query_n in range(dist_arr_isorted.shape[1]):

        # return "true" video ID's in df
        video_id_idx = returnVideoID_index(df, df_eval, query_n)

        # evaluate the ranking of the "true" video ID
        true_rank = np.argwhere(dist_arr_isorted[:,query_n]==video_id_idx)[0][0]

        # store the "true" video ID's ranking in array
        true_rank_arr[0,query_n] = true_rank

    return true_rank_arr

In [15]:
# initialize distance metrics to experiment
dist_name_list = ['euclidean', 'manhattan', 'chebyshev']
sim_name_list = ['cos_sim', 'dot_score']

In [16]:
# evaluate all possible combinations of model, columns to embed, and distance metrics

# initialize list to store results
eval_results = []

# loop through all models
for model_name in model_name_list:

    # generate query embedding
    query_embedding = query_embedding_dict[model_name]

    # loop through text columns
    for column_name in column_to_embed_list:

        # generate column embedding
        embedding_arr = text_embedding_dict[model_name+'_'+column_name]

        # loop through distance metrics
        for dist_name in dist_name_list:

            # compute distance between video text and query
            dist = DistanceMetric.get_metric(dist_name)
            dist_arr = dist.pairwise(embedding_arr, query_embedding)

            # sort indexes of distance array
            dist_arr_isorted = np.argsort(dist_arr, axis=0)

            # define label for search method
            method_name = "_".join([model_name, column_name, dist_name])

            # evaluate the ranking of the ground truth
            true_rank_arr = evalTrueRankings(dist_arr_isorted, df, df_eval)

            # store results
            eval_list = [method_name] + true_rank_arr.tolist()[0]
            eval_results.append(eval_list)

        # loop through sbert similarity scores
        for sim_name in sim_name_list:
            # apply similarity score from sbert
            cmd = "dist_arr = -util." + sim_name + "(embedding_arr, query_embedding)"
            exec(cmd)

            # sort indexes of distance array (notice minus sign in front of cosine similarity)
            dist_arr_isorted = np.argsort(dist_arr, axis=0)

            # define label for search method
            method_name = "_".join([model_name, column_name, sim_name.replace("_","-")])

            # evaluate the ranking of the ground truth
            true_rank_arr = evalTrueRankings(dist_arr_isorted, df, df_eval)

            # store results
            eval_list = [method_name] + true_rank_arr.tolist()[0]
            eval_results.append(eval_list)

In [17]:
# compute rankings for title + transcripts embedding
for model_name in model_name_list:

    # generate embeddings
    embedding_arr1 = text_embedding_dict[model_name+'_title']
    embedding_arr2 = text_embedding_dict[model_name+'_transcript']
    query_embedding = query_embedding_dict[model_name]

    for dist_name in dist_name_list:

        # compute distance between video text and query
        dist = DistanceMetric.get_metric(dist_name)
        dist_arr = dist.pairwise(embedding_arr1, query_embedding) + dist.pairwise(embedding_arr2, query_embedding)

        # sort indexes of distance array
        dist_arr_isorted = np.argsort(dist_arr, axis=0)

         # define label for search method
        method_name = "_".join([model_name, "title-transcript", dist_name])

        # evaluate the ranking of the ground truth
        true_rank_arr = evalTrueRankings(dist_arr_isorted, df, df_eval)

        # store results
        eval_list = [method_name] + true_rank_arr.tolist()[0]
        eval_results.append(eval_list)

    # loop through sbert similarity scores
    for sim_name in sim_name_list:
        # apply similarity score from sbert
        cmd = "dist_arr = -util." + sim_name + "(embedding_arr1, query_embedding) - util."+ sim_name + "(embedding_arr2, query_embedding)"
        exec(cmd)

        # sort indexes of distance array (notice minus sign in front of cosine similarity)
        dist_arr_isorted = np.argsort(dist_arr, axis=0)

        # define label for search method
        method_name = "_".join([model_name, "title-transcript", sim_name.replace("_","-")])

        # evaluate the ranking of the ground truth
        true_rank_arr = evalTrueRankings(dist_arr_isorted, df, df_eval)

        # store results
        eval_list = [method_name] + true_rank_arr.tolist()[0]
        eval_results.append(eval_list)

In [18]:
len(eval_results)

45

In [19]:
# define schema for results dataframe
schema_dict = {'method_name':str}
for i in range(len(eval_results[0])-1):
    schema_dict['rank_query-'+str(i)] = float

# store results in dataframe
df_results = pl.DataFrame(eval_results, schema=schema_dict)
df_results.head()

/usr/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


method_name,rank_query-0,rank_query-1,rank_query-2,rank_query-3,rank_query-4,rank_query-5,rank_query-6,rank_query-7,rank_query-8,rank_query-9,rank_query-10,rank_query-11,rank_query-12,rank_query-13,rank_query-14,rank_query-15,rank_query-16,rank_query-17,rank_query-18,rank_query-19,rank_query-20,rank_query-21,rank_query-22,rank_query-23,rank_query-24,rank_query-25,rank_query-26,rank_query-27,rank_query-28,rank_query-29,rank_query-30,rank_query-31,rank_query-32,rank_query-33,rank_query-34,rank_query-35,rank_query-36,rank_query-37,rank_query-38,rank_query-39,rank_query-40,rank_query-41,rank_query-42,rank_query-43,rank_query-44,rank_query-45,rank_query-46,rank_query-47,rank_query-48,rank_query-49,rank_query-50,rank_query-51,rank_query-52,rank_query-53,rank_query-54,rank_query-55,rank_query-56,rank_query-57,rank_query-58,rank_query-59,rank_query-60,rank_query-61,rank_query-62,rank_query-63
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""all-MiniLM-L6-v2_title_euclide…",0.0,1.0,21.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
"""all-MiniLM-L6-v2_title_manhatt…",0.0,1.0,10.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,8.0,1.0,0.0,0.0,1.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,12.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
"""all-MiniLM-L6-v2_title_chebysh…",0.0,3.0,51.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,41.0,0.0,0.0,5.0,74.0,0.0,3.0,0.0,33.0,0.0,0.0,0.0,15.0,8.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0,1.0,0.0,0.0,1.0,0.0,8.0,9.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,5.0,0.0,94.0,17.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
"""all-MiniLM-L6-v2_title_cos-sim""",0.0,1.0,21.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
"""all-MiniLM-L6-v2_title_dot-sco…",0.0,1.0,21.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10.0,1.0,0.0,0.0,1.0,0.0,6.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [20]:
# compute mean rankings of ground truth search result
df_results = df_results.with_columns(new_col=pl.mean_horizontal(df_results.columns[1:])).rename({"new_col": "rank_query-mean"})

In [21]:
# compute number of ground truth results which appear in top 3
for i in [1,3]:
    df_results = df_results.with_columns(new_col=pl.sum_horizontal(df_results[:,1:-1]<i)).rename({"new_col": "num_in_top-"+str(i)})

LOOK AT TOP RESULTS

In [22]:
df_summary = df_results[['method_name', "rank_query-mean", "num_in_top-1", "num_in_top-3"]]

In [23]:
print(df_summary.sort('rank_query-mean').head())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────┬──────────────┬──────────────┐
│ method_name                     ┆ rank_query-mean ┆ num_in_top-1 ┆ num_in_top-3 │
│ ---                             ┆ ---             ┆ ---          ┆ ---          │
│ str                             ┆ f64             ┆ u32          ┆ u32          │
╞═════════════════════════════════╪═════════════════╪══════════════╪══════════════╡
│ multi-qa-mpnet-base-dot-v1_tit… ┆ 1.15625         ┆ 37           ┆ 58           │
│ all-MiniLM-L6-v2_title-transcr… ┆ 1.203125        ┆ 37           ┆ 57           │
│ all-MiniLM-L6-v2_title-transcr… ┆ 1.25            ┆ 37           ┆ 58           │
│ all-MiniLM-L6-v2_title_manhatt… ┆ 1.28125         ┆ 41           ┆ 57           │
│ all-MiniLM-L6-v2_title-transcr… ┆ 1.328125        ┆ 37           ┆ 57           │
└─────────────────────────────────┴─────────────────┴──────────────┴──────────────┘


In [24]:
df_summary.sort('rank_query-mean').head()[0,0]

'multi-qa-mpnet-base-dot-v1_title-transcript_dot-score'

In [25]:
print(df_summary.sort("num_in_top-1", descending=True).head())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────┬──────────────┬──────────────┐
│ method_name                     ┆ rank_query-mean ┆ num_in_top-1 ┆ num_in_top-3 │
│ ---                             ┆ ---             ┆ ---          ┆ ---          │
│ str                             ┆ f64             ┆ u32          ┆ u32          │
╞═════════════════════════════════╪═════════════════╪══════════════╪══════════════╡
│ multi-qa-mpnet-base-dot-v1_tit… ┆ 2.171875        ┆ 43           ┆ 56           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
│ all-MiniLM-L6-v2_title_euclide… ┆ 1.5             ┆ 41           ┆ 57           │
└─────────────────────────────────┴─────────────────┴──────────────┴──────────────┘


In [26]:
df_summary.sort("num_in_top-1", descending=True).head()[0,0]

'multi-qa-mpnet-base-dot-v1_title-transcript_manhattan'

In [27]:
print(df_summary.sort("num_in_top-3", descending=True).head())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────┬──────────────┬──────────────┐
│ method_name                     ┆ rank_query-mean ┆ num_in_top-1 ┆ num_in_top-3 │
│ ---                             ┆ ---             ┆ ---          ┆ ---          │
│ str                             ┆ f64             ┆ u32          ┆ u32          │
╞═════════════════════════════════╪═════════════════╪══════════════╪══════════════╡
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.0625          ┆ 41           ┆ 60           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 41           ┆ 60           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
│ multi-qa-distilbert-cos-v1_tit… ┆ 2.234375        ┆ 42           ┆ 59           │
└─────────────────────────────────┴─────────────────┴──────────────┴──────────────┘


In [28]:
df_summary.sort("num_in_top-3", descending=True).head()[0,0]

'multi-qa-distilbert-cos-v1_title-transcript_euclidean'

In [29]:
for i in range(4):
    print(df_summary.sort("num_in_top-3", descending=True)['method_name'][i])

multi-qa-distilbert-cos-v1_title-transcript_euclidean
multi-qa-distilbert-cos-v1_title-transcript_manhattan
multi-qa-distilbert-cos-v1_title_euclidean
multi-qa-distilbert-cos-v1_title_cos-sim
